In [2]:
import pandas_ta as ta
help(ta.psar)

['ALL_PATTERNS',
 'AllStrategy',
 'AnalysisIndicators',
 'BasePandasObject',
 'CANGLE_AGG',
 'CDL_PATTERN_NAMES',
 'Category',
 'CommonStrategy',
 'DataFrame',
 'DistributionNotFound',
 'EXCHANGE_TZ',
 'Imports',
 'List',
 'Optional',
 'PandasObject',
 'Path',
 'Pool',
 'RATE',
 'Series',
 'Strategy',
 'Timedelta',
 'Timestamp',
 'Tuple',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_dist',
 'aberration',
 'above',
 'above_value',
 'accbands',
 'ad',
 'adosc',
 'adx',
 'alma',
 'alphavantage',
 'amat',
 'ao',
 'aobv',
 'apo',
 'argmax',
 'argmin',
 'aroon',
 'atr',
 'av',
 'bbands',
 'below',
 'below_value',
 'bias',
 'bop',
 'brar',
 'cagr',
 'calmar_ratio',
 'candle_color',
 'candles',
 'category_files',
 'cci',
 'cdl',
 'cdl_doji',
 'cdl_inside',
 'cdl_pattern',
 'cdl_z',
 'cfo',
 'cg',
 'chop',
 'cksp',
 'cmf',
 'cmo',
 'combination',
 'coppock',
 'core',
 'cpu_count',
 'cross',
 'cr

In [1]:
# Importer les modules nécessaires
import os
import sys
import pandas as pd
from google.cloud import storage
import pandas_ta as ta
# Ajouter le chemin de base de ton projet au PYTHONPATH
sys.path.append("/root/EosData/Cicada-binance")

# Importer le EnvController pour la gestion de l'environnement
from src.commons.env_manager.env_controller import EnvController

# Initialiser le EnvController pour charger les configurations
env_controller = EnvController(config_base_path="/root/EosData/Cicada-binance/config")

# Récupérer le nom du bucket depuis les configurations YAML
bucket_name = env_controller.get_yaml_config('Ta-lib', 'bucket')
print(f"Nom du bucket: {bucket_name}")

# Créer un client Google Cloud Storage
client = storage.Client()

# Accéder au bucket GCS
bucket = client.bucket(bucket_name)

Loading shared env from: /root/EosData/Cicada-binance/config/shared/.env.shared
Loading environment-specific env from: /root/EosData/Cicada-binance/config/development/secrets.env
Loading shared YAML config from: /root/EosData/Cicada-binance/config/shared/db-config.yaml
Loading environment-specific YAML config from: /root/EosData/Cicada-binance/config/development/app-config.yaml
Loading YAML file: /root/EosData/Cicada-binance/config/development/app-config.yaml
Nom du bucket: production-trustia-raw-data


In [2]:
def load_gcs_file_to_dataframe(bucket_name, file_path, file_format='parquet'):
    """
    Télécharge un fichier à partir de Google Cloud Storage et le charge dans un DataFrame.

    :param bucket_name: Le nom du bucket GCS.
    :param file_path: Le chemin du fichier dans le bucket.
    :param file_format: Le format du fichier ('parquet', 'csv', etc.).
    :return: DataFrame contenant les données du fichier.
    """
    try:
        # Créer un client GCS
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Récupérer le blob (fichier) depuis le bucket
        blob = bucket.blob(file_path)
        
        # Créer un fichier temporaire local
        temp_file_path = '/tmp/temp_file.' + file_format
        
        # Télécharger le fichier depuis GCS
        blob.download_to_filename(temp_file_path)
        print(f"Fichier téléchargé depuis GCS : {temp_file_path}")
        
        # Charger le fichier dans un DataFrame
        if file_format == 'parquet':
            df = pd.read_parquet(temp_file_path)
        elif file_format == 'csv':
            df = pd.read_csv(temp_file_path)
        else:
            raise ValueError(f"Format de fichier non supporté : {file_format}")
        
        # Retourner le DataFrame
        return df
    
    except Exception as e:
        print(f"Erreur lors du téléchargement ou du chargement du fichier : {e}")
        return None


In [3]:
def read_parquet_with_pyarrow(self, file_path):
    try:
        table = pq.read_table(file_path)
        df = table.to_pandas()
        df.columns = [
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_volume', 'count', 'taker_buy_volume',
            'taker_buy_quote_volume', 'ignore'
        ]
        return df
    except Exception as e:
        self.logger.log_error(f"Error reading parquet file with pyarrow: {e}")
        return pd.DataFrame()

# OUTPUT

In [ ]:
# Spécifier le chemin du fichier dans le bucket (sans le nom du bucket)
gcs_file_path = "Transformed/cryptos/BTCUSDT/spot/bars/time-bars/4h/data_2023_01_01_2024_10_21.parquet"  # Remplace par le chemin correct du fichier

# Charger le fichier GCS dans un DataFrame
df = load_gcs_file_to_dataframe(bucket_name, gcs_file_path, file_format='parquet')

# Afficher le DataFrame
if df is not None:
    print("DataFrame chargé avec succès :")
    display(df)
else:
    print("Le DataFrame n'a pas pu être chargé.")


Erreur lors du téléchargement ou du chargement du fichier : 404 GET https://storage.googleapis.com/download/storage/v1/b/production-trustia-raw-data/o/Transformed%2Fcryptos%2FBTCUSDT%2Fspot%2Fbars%2Ftime-bars%2F1h%2Fdata_2023_01_01_2024_10_21.parquet?alt=media: No such object: production-trustia-raw-data/Transformed/cryptos/BTCUSDT/spot/bars/time-bars/1h/data_2023_01_01_2024_10_21.parquet: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
Le DataFrame n'a pas pu être chargé.


In [5]:
# Spécifier le chemin du fichier dans le bucket (sans le nom du bucket)
gcs_file_path = "Transformed/cryptos/BTCUSDT/spot/bars/time-bars/4h/data_2023_01_01_2024_10_21.parquet"  # Remplace par le chemin correct du fichier

# Charger le fichier GCS dans un DataFrame
df = load_gcs_file_to_dataframe(bucket_name, gcs_file_path, file_format='parquet')

# Afficher le DataFrame
if df is not None:
    print("DataFrame chargé avec succès :")
    display(df)
else:
    print("Le DataFrame n'a pas pu être chargé.")


Fichier téléchargé depuis GCS : /tmp/temp_file.parquet


DataFrame chargé avec succès :


,timestamp,open,high,low,close,volume,EMA_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,WILLR_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,2023-01-01 04:00:00,16533.04,16550.00,16499.01,16526.19,16532.24115,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0
1,2023-01-01 08:00:00,16525.70,16557.00,16505.20,16556.66,15915.96701,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16499.010000,NaN,0.04,0
2,2023-01-01 12:00:00,16556.66,16572.94,16533.68,16558.73,15046.09096,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16499.010000,NaN,0.06,0
3,2023-01-01 16:00:00,16558.73,16623.65,16558.00,16603.08,18532.64857,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16503.445800,NaN,0.08,0
4,2023-01-01 20:00:00,16603.53,16628.00,16586.24,16616.75,15382.64278,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16513.062136,NaN,0.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,2024-10-21 04:00:00,68940.00,69185.18,68488.33,68514.00,2657.04256,68040.177774,NaN,NaN,NaN,...,-50.418542,66999.395205,68143.498,69287.600795,3.357922,0.661918,68016.245384,NaN,0.08,0
6596,2024-10-21 08:00:00,68513.99,68661.00,68119.01,68238.01,4244.21652,68059.018939,NaN,NaN,NaN,...,-83.596403,67227.405703,68211.777,69196.148297,2.886221,0.513325,NaN,69400.000000,0.02,1
6597,2024-10-21 12:00:00,68238.00,68328.68,66840.67,67341.02,12814.65836,67990.638087,NaN,NaN,NaN,...,-80.449962,67178.094758,68199.295,69220.495242,2.994753,0.079771,NaN,69374.380200,0.04,0
6598,2024-10-21 16:00:00,67341.02,67941.17,66894.00,67763.00,5541.16575,67968.958269,NaN,NaN,NaN,...,-63.962053,67407.074718,68245.445,69083.815282,2.456927,0.212272,NaN,69273.031792,0.04,0


In [5]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'EMA_20',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'SMA_50', 'WMA_30',
       'STOCHk_14_3_3', 'STOCHd_14_3_3', 'ADX_14', 'DMP_14', 'DMN_14',
       'ATRr_14', 'CCI_20_0.015', 'ROC_12', 'MFI_14', 'TRIX_15_9',
       'TRIXs_15_9', 'OBV', 'WILLR_14'],
      dtype='object')

# INPUT

In [6]:
# Spécifier le chemin du fichier dans le bucket (sans le nom du bucket)
gcs_file_path = "Raw/binance-data-vision/historical/BTCUSDT/futures/klines/15m/2024/10/21/data.parquet"  # Remplace par le chemin correct du fichier

# Charger le fichier GCS dans un DataFrame
df = load_gcs_file_to_dataframe(bucket_name, gcs_file_path, file_format='parquet')

# Afficher le DataFrame
if df is not None:
    print("DataFrame chargé avec succès :")
    display(df)
else:
    print("Le DataFrame n'a pas pu être chargé.")

Fichier téléchargé depuis GCS : /tmp/temp_file.parquet
DataFrame chargé avec succès :


,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1729468800000,69020.2,69125.0,68957.2,69124.9,1453.355,1729469699999,1.003387e+08,26153,737.136,5.089169e+07,0
1,1729469700000,69124.9,69566.0,69124.9,69400.3,7388.546,1729470599999,5.127013e+08,85677,4163.723,2.889364e+08,0
2,1729470600000,69400.3,69555.0,69233.0,69254.7,2546.768,1729471499999,1.766399e+08,43848,1226.380,8.507234e+07,0
3,1729471500000,69254.7,69254.7,69055.0,69153.9,2292.736,1729472399999,1.585320e+08,31378,902.957,6.243258e+07,0
4,1729472400000,69153.9,69287.5,69091.2,69283.0,1537.888,1729473299999,1.063843e+08,24784,912.562,6.313607e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...
91,1729550700000,67674.5,67793.4,67674.4,67769.9,1320.284,1729551599999,8.944530e+07,19473,888.448,6.018838e+07,0
92,1729551600000,67769.9,67770.0,67408.7,67530.5,4214.968,1729552499999,2.847166e+08,42836,1705.069,1.151686e+08,0
93,1729552500000,67530.4,67574.1,67475.3,67489.9,1020.234,1729553399999,6.888789e+07,18723,567.303,3.830541e+07,0
94,1729553400000,67489.9,67521.1,67400.0,67513.8,1077.977,1729554299999,7.272442e+07,20574,583.887,3.939345e+07,0
